In [84]:
import os
import time
from functools import reduce
import numpy as np
import pandas as pd

In [85]:
load_data = pd.read_excel("基金前十持有人.xlsx",  usecols="A,B,C,D,E,F,G,H,I,J,K,L,M")
mask = load_data["证券简称"].str.contains("ETF", case=False).fillna(False)  
  
# 使用布尔掩码来过滤DataFrame  
load_data = load_data[mask] 

load_data

C:\Users\zhc12\AppData\Local\Temp\ipykernel_13820\1860390179.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mask = load_data["证券简称"].str.contains("ETF", case=False).fillna(False)


,证券代码,证券简称,第1名持有人名称,第2名持有人名称,第3名持有人名称,第4名持有人名称,第5名持有人名称,第6名持有人名称,第7名持有人名称,第8名持有人名称,第9名持有人名称,第10名持有人名称,第11名持有人名称
4,501302.OF,南方恒生ETF联接A,朱晓颖,湛胜仁,深圳市老龄事业发展基金会,邵常红,骆念春,林小红,钱晓健,徐静凤,吴强,林淑娟,NaN
6,517360.OF,华安中证沪港深科技100ETF,朱淑杰,中信证券股份有限公司,中国国际金融股份有限公司,陆东强,海通证券股份有限公司,中信建投证券股份有限公司,沈根先,许志琼,郝和兰,杜微微,NaN
11,513990.OF,招商上证港股通ETF,朱洁,方正证券股份有限公司,BARCLAYSBANKPLC,吴震,中信证券股份有限公司,刘俊刚,邵峥,招商证券股份有限公司,吴伟鸿,中国平安人寿保险股份有限公司-自有资金,NaN
14,511010.OF,国泰上证5年期国债ETF,铸锋资产管理(北京)有限公司-铸锋国锋6号私募证券投资基金,宁波梅山保税港区凌顶投资管理有限公司-凌顶泰山九号私募证券投资基金,铸锋资产管理(北京)有限公司-铸锋长锋11号私募证券投资基金,量桥投资管理(上海)有限公司-量桥投资兴泰十号私募证券投资基金,宁波梅山保税港区凌顶投资管理有限公司-凌顶北斗六号私募证券投资基金,铸锋资产管理(北京)有限公司-铸锋国锋9号私募证券投资基金,铸锋资产管理(北京)有限公司-铸锋赤霄3号私募证券投资基金,铸锋资产管理(北京)有限公司-铸锋国锋1号私募证券投资基金,北京尚艺投资管理有限公司-尚艺阳光13号私募证券投资基金,上海稳博投资管理有限公司-稳博戊申1号私募证券投资基金,NaN
21,159901.OF,易方达深证100ETF,中央汇金资产管理有限责任公司,中国银行股份有限公司-易方达深证100交易型开放式指数证券投资基金联接基金,易方达基金-中央汇金资产管理有限责任公司-易方达基金-汇金资管单一资产管理计划,中国平安人寿保险股份有限公司-自有资金,中国平安人寿保险股份有限公司-分红-个险分红,邱莉莎,中荷人寿保险有限公司,肖鹏,中央汇金投资有限责任公司,东方证券股份有限公司,浙江永安资本管理有限公司
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266,513160.OF,银华恒生港股通中国科技ETF,BARCLAYSBANKPLC,长江财产保险股份有限公司-委托民生银行组合,深圳竹芒科技有限公司,方正证券股份有限公司,中信建投证券股份有限公司,中信证券股份有限公司,中国平安人寿保险股份有限公司-分红-个险分红,南京盛泉恒元投资有限公司-盛泉恒元量化套利专项24号私募证券投资基金,中国平安人寿保险股份有限公司-自有资金,国朝冲,NaN
1267,516330.OF,华泰柏瑞中证物联网ETF,BARCLAYSBANKPLC,北京恒德时代私募基金管理有限公司-恒德利得2号私募证券投资基金,广发证券股份有限公司,中信证券股份有限公司,华泰证券股份有限公司,中国国际金融股份有限公司,魏柳凤,海通证券股份有限公司,欧康,于进进,NaN
1268,513870.OF,富国纳斯达克100ETF,BARCLAYSBANKPLC,安然,张小涛,上海橡杉资产管理有限公司,中信建投证券股份有限公司,佟德鑫,上海橡杉资产管理有限公司-橡杉量化套利六号私募证券投资基金,林形省,曹祎恺,李梦龙,NaN
1270,513200.OF,易方达中证港股通医药卫生综合ETF,百年人寿保险股份有限公司-传统保险产品,新华人寿保险股份有限公司,信泰人寿保险股份有限公司-分红产品,BARCLAYSBANKPLC,中国光大银行股份有限公司-易方达中证港股通医药卫生综合交易型开放式指数证券投资基金发起式联接基金,中国民生银行股份有限公司-汇添富添福盈和稳健养老目标一年持有期混合型基金中基金,上海浦东发展银行股份有限公司-汇添富聚焦经典一年持有期混合型基金中基金(FOF,工银安盛人寿保险有限公司,百年人寿保险股份有限公司-平衡自营,新华资管-工商银行-新华资产-明仁八号资产管理产品,易方达基金管理有限公司


In [86]:
# 创建空字典用于存储结果
holder_to_security_dict = {}

# 遍历 DataFrame，忽略最后一行（数据来源）
for i in range(len(load_data) - 1):
    row = load_data.iloc[i]
    security_short_name = row["证券简称"]
    holder_names = row[3:]  # 从第1名持有人名称开始

    for holder_name in holder_names:
        if pd.isna(holder_name):  # 跳过当前持有人名称，如果它是 NaN
            continue
        if len(holder_name) < 5: #过滤到个人
            continue
        if holder_name in holder_to_security_dict:
            holder_to_security_dict[holder_name].append(str(security_short_name))  # 将证券简称转换为字符串
        else:
            holder_to_security_dict[holder_name] = [str(security_short_name)]  # 将证券简称转换为字符串

data = {
    '一级分类':[],
    '二级分类':[],
    '序号':[],
    '主体': [],
    'ETF': []
}

number = 1
 
for name in holder_to_security_dict.keys():
    for value in holder_to_security_dict[name]:
        data['序号'].append(number)
        data['主体'].append(name)
        data['ETF'].append(value) 
        if "风险管理" in name or "苏州创元和" in name or "兴全基金-兴业银行-兴业证券股份有限公司" in name:
            data['一级分类'].append("暂无")
            data['二级分类'].append("暂无") 
            continue    
        if ("私募" in name or '量游' in name or '财富投资' in name or '永安资本' in name or '诚通金控' in name or '杭州瑞峰' in name
         or "资管有限公司" in name or "投资有限公司" in name or "投资管理有限公司" in name or "企业管理咨询" in name or "投资咨询" in name
         or "资产管理有限公司" in name or "资产管理合伙企业" in name or "投资公司" in name or "经营公司" in name or "投资中心" in name
         or "商务咨询中心" in name or "股权投资" in name or "投资管理合伙企业" in name or "资本管理有限公司" in name) and (
            "人寿" not in name and "永赢" not in name and "保险" not in name and "中国国际金融" not in name): 
            data['一级分类'].append("私募基金")
            data['二级分类'].append("私募产品")
            continue
        if "FOF" in name:
            data['一级分类'].append("FOF")
            data['二级分类'].append("FOF产品") 
            continue
        if "联接基金" in name or "ETF联接" in name or "ETF发起式联接" in name or "基金联接" in name:
            data['一级分类'].append("公募基金")
            data['二级分类'].append("联接基金产品")
            continue
        # if "资产管理计划" in name:
            # if '基金' in name: 
            #     data['一级分类'].append("公募基金")
            #     data['二级分类'].append("公募资管计划") 
            # elif '期货' in name: 
            #     data['一级分类'].append("期货公司")
            #     data['二级分类'].append("期货资管计划")
            # elif '中信里昂资管' in name or '光大永明资管' in name or '集合资产管理计划' in name or '证券资管' in name or '资产管理产品' in name: 
            #     data['一级分类'].append("券商资管")
            #     data['二级分类'].append("券商资管计划")
            # else:
            #     data['一级分类'].append("暂无")
            #     data['二级分类'].append("暂无")
            # continue  
        if "资产管理" in name or '集合资产' in name or '证券资管' in name or '资产管理产品' in name or "太平洋资管" in name or "太平资管" in name or "中国人寿资产管理" in name:
            data['一级分类'].append("资管")
            data['二级分类'].append("资管产品")
            continue  
        if "基金管理有限公司" in name or "基金管理股份有限公司" in name: 
            data['一级分类'].append("公募基金")
            data['二级分类'].append("公募基金产品")
            continue  
        if "期货" in name:
            data['一级分类'].append("期货公司")
            data['二级分类'].append("期货产品")
            continue  
        if "证券股份" in name or "证券有限公司" in name or "中国国际金融" in name or "证券有限责任公司" in name or '中信中证资本' in name: 
            data['一级分类'].append("证券公司")
            data['二级分类'].append("证券产品")
            continue
        if "保险" in name or '人寿资产管理' in name:
            data['一级分类'].append("保险")
            data['二级分类'].append("保险产品")
            continue
        if "信托" in name:
            data['一级分类'].append("信托")
            data['二级分类'].append("信托产品")
            continue
        if "理财" in name:
            data['一级分类'].append("银行")
            data['二级分类'].append("银行理财产品")
            continue
        if "养老金" in name or "养老" in name:
            data['一级分类'].append("公募基金")
            data['二级分类'].append("养老金产品") 
            continue          
        if "年金" in name:
            data['一级分类'].append("年金")
            data['二级分类'].append("年金产品") 
            continue
        if ("BARCLAY" in name or  '三菱日联' in name or 'UBSAG' in name or 'GICPRIVATELIMITED' in name or '野村新加坡' in name
         or "上海汇丰" in name or "星展" in name or "大和资产管理" in name):
            data['一级分类'].append("外资")
            data['二级分类'].append("外资产品") 
            continue
        if "银行" in name: 
            data['一级分类'].append("银行")
            data['二级分类'].append("银行产品")
            continue
        if ("有限公司" in name or "有限责任公司" in name or "工程管理" in name or "大鹏大坑" in name) and "中央" not in name: 
            data['一级分类'].append("产业")
            data['二级分类'].append("产业产品")
            continue
        else:
            data['一级分类'].append("暂无")
            data['二级分类'].append("暂无")
        
    number+=1
df = pd.DataFrame(data)
df.to_excel("ETF_all_new.xlsx",index=True)
    

In [87]:
# 假设您的 DataFrame 名为 load_data

# ...（之前的代码，构建 holder_to_security_dict 字典）
# ...（将字典转换为 DataFrame 的代码）

# licai_df = holder_df.loc[:, [col for col in holder_df.columns if '理财' in col]].T

# insurance_df = holder_df.loc[:, [col for col in holder_df.columns if '保险' in col]].T

# trust_df = holder_df.loc[:, [col for col in holder_df.columns if '信托' in col]].T

# security_df = holder_df.loc[:, [col for col in holder_df.columns if '证券股份' in col]].T

# MF_df = holder_df.loc[:, [col for col in holder_df.columns if '联接基金' in col]].T

# PE_df = holder_df.loc[:, [col for col in holder_df.columns if '私募' in col]].T

# futures_df = holder_df.loc[:, [col for col in holder_df.columns if '期货' in col]].T

# AM_df =  holder_df.loc[:, [col for col in holder_df.columns if '资产管理' in col and '基金' in col and '私募' not in col and ('银行' in col or '证券' in col)]].T

# pension_df = holder_df.loc[:, [col for col in holder_df.columns if '养老金' in col]].T

# FOF_df = holder_df.loc[:, [col for col in holder_df.columns if 'FOF' in col]].T

# yearfund_df = holder_df.loc[:, [col for col in holder_df.columns if '年金' in col]].T

# company_df = holder_df.loc[:, [col for col in holder_df.columns if '私募' not in col and 
# '证券' not in col and '保险' not in col and '资产管理' not in col and '年金' not in col and 
# 'FOF' not in col and '私募' not in col and '信托' not in col and '理财' not in col and
# '养老金' not in col and '投资' not in col and '银行' not in col and '资本' not in col and
# '财富' not in col and '期货' not in col and '基金' not in col and 'LOF' not in col
# ]].T

# with pd.ExcelWriter('holder_securities.xlsx') as writer:

#     holder_df.to_excel(writer, sheet_name='总表', index=False)

#     AM_df.to_excel(writer, sheet_name='公募资产专户', index=False)

#     FOF_df.to_excel(writer, sheet_name='公募FOF', index=False)

#     pension_df.to_excel(writer, sheet_name='公募养老金', index=False)

#     MF_df.to_excel(writer, sheet_name='联接基金', index=False)


#     licai_df.to_excel(writer, sheet_name='理财', index=False)

#     PE_df.to_excel(writer, sheet_name='私募', index=False)

#     trust_df.to_excel(writer, sheet_name='信托', index=False)

#     futures_df.to_excel(writer, sheet_name='期货', index=False)

#     insurance_df.to_excel(writer, sheet_name='保险', index=False)

#     security_df.to_excel(writer, sheet_name='证券', index=False)

#     yearfund_df.to_excel(writer, sheet_name='年金', index=False)

#     company_df.to_excel(writer, sheet_name='产业', index=False)

# 关闭 Excel Writer，保存文件